# Authentication Anomaly Detections

## Phase
Phase 2 — Identity Threat Detection

## Objective
Identify suspicious login behaviors using identity context and behavioral baselines.


In [1]:
import pandas as pd
from pathlib import Path


In [2]:
PROJECT_ROOT = Path(r"D:\soc-dashboard-suite-main\soc-dashboard-suite-main")

INPUT_PATH = PROJECT_ROOT / "data" / "enriched" / "authentication_with_context.csv"
OUTPUT_PATH = PROJECT_ROOT / "data" / "enriched" / "auth_alerts.csv"

auth_df = pd.read_csv(INPUT_PATH, parse_dates=["event_time"])
auth_df.sort_values(["user_id", "event_time"], inplace=True)

auth_df.head()


,event_time,user_id,user_role,source_ip,source_country,device_id,login_status,event_type,hour,date,baseline_hour,hour_std,hour_deviation,unusual_login_hour,first_seen_country,new_login_country,failed_login
250,2025-01-01 11:59:00,user1,user,192.168.220.20,IN,device_5,failed,authentication,11,2025-01-01,12.0,4.752046,1.0,False,IN,False,True
3202,2025-01-01 13:26:00,user1,user,192.168.236.46,UK,device_16,failed,authentication,13,2025-01-01,12.0,4.752046,1.0,False,IN,True,True
4882,2025-01-01 13:30:00,user1,user,192.168.40.179,US,device_14,success,authentication,13,2025-01-01,12.0,4.752046,1.0,False,IN,True,False
4470,2025-01-01 15:32:00,user1,user,192.168.42.197,IN,device_9,success,authentication,15,2025-01-01,12.0,4.752046,3.0,False,IN,False,False
3856,2025-01-01 17:59:00,user1,user,192.168.107.205,IN,device_10,success,authentication,17,2025-01-01,12.0,4.752046,5.0,False,IN,False,False


In [3]:
privileged_roles = ["admin", "executive"]

unusual_privileged_login = auth_df[
    (auth_df["user_role"].isin(privileged_roles)) &
    (auth_df["unusual_login_hour"] == True) &
    (auth_df["login_status"].str.lower() == "success")
].copy()

unusual_privileged_login["detection_id"] = "AUTH_UNUSUAL_PRIV_LOGIN"
unusual_privileged_login["severity"] = "high"
unusual_privileged_login["alert_reason"] = "Privileged user logged in at unusual hour"


In [4]:
new_country_login = auth_df[
    (auth_df["new_login_country"] == True) &
    (auth_df["login_status"].str.lower() == "success")
].copy()

new_country_login["detection_id"] = "AUTH_NEW_COUNTRY_LOGIN"
new_country_login["severity"] = "high"
new_country_login["alert_reason"] = "Login from new country"


In [5]:
auth_df["prev_failed"] = auth_df.groupby("user_id")["failed_login"].shift(1)

suspicious_success = auth_df[
    (auth_df["prev_failed"] == True) &
    (auth_df["login_status"].str.lower() == "success")
].copy()

suspicious_success["detection_id"] = "AUTH_SUCCESS_AFTER_FAIL"
suspicious_success["severity"] = "medium"
suspicious_success["alert_reason"] = "Successful login after recent failure"


In [6]:
auth_alerts = pd.concat([
    unusual_privileged_login,
    new_country_login,
    suspicious_success
], ignore_index=True)

auth_alerts = auth_alerts[[
    "event_time",
    "user_id",
    "user_role",
    "source_country",
    "device_id",
    "login_status",
    "detection_id",
    "severity",
    "alert_reason"
]]

auth_alerts.head()


,event_time,user_id,user_role,source_country,device_id,login_status,detection_id,severity,alert_reason
0,2025-01-02 00:25:00,user1,admin,US,device_6,success,AUTH_UNUSUAL_PRIV_LOGIN,high,Privileged user logged in at unusual hour
1,2025-01-13 02:05:00,user10,executive,DE,device_19,success,AUTH_UNUSUAL_PRIV_LOGIN,high,Privileged user logged in at unusual hour
2,2025-01-04 01:21:00,user11,executive,UK,device_10,success,AUTH_UNUSUAL_PRIV_LOGIN,high,Privileged user logged in at unusual hour
3,2025-01-20 03:38:00,user11,admin,DE,device_6,success,AUTH_UNUSUAL_PRIV_LOGIN,high,Privileged user logged in at unusual hour
4,2025-01-23 00:44:00,user13,executive,DE,device_14,success,AUTH_UNUSUAL_PRIV_LOGIN,high,Privileged user logged in at unusual hour


In [7]:
auth_alerts.to_csv(OUTPUT_PATH, index=False)
print("Saved authentication alerts to:", OUTPUT_PATH)


Saved authentication alerts to: D:\soc-dashboard-suite-main\soc-dashboard-suite-main\data\enriched\auth_alerts.csv
